In [5]:
import os
import cv2
import numpy as np

# Paths
input_folder  = r"C:\Users\Hardika\Desktop\NNpro1_7species\unceen"
output_folder = r"C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves"
os.makedirs(output_folder, exist_ok=True)

# Augmentation settings
rotation_angles = [0, 90, 180]
contrast_factors = [1.0, 1.3]

# Loop through images
for fname in os.listdir(input_folder):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        print(f"Skipping unsupported file: {fname}")
        continue

    fpath = os.path.join(input_folder, fname)
    img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Failed to read: {fpath}")
        continue

    # Resize
    img = cv2.resize(img, (256, 256))

    # Thresholding and cleanup
    th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)
    inv = cv2.bitwise_not(th)
    cnts, _ = cv2.findContours(inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    leaf_mask = np.zeros_like(th)
    for c in cnts:
        if cv2.contourArea(c) > 500:
            cv2.drawContours(leaf_mask, [c], -1, 255, -1)
    leaf_mask_inv = cv2.bitwise_not(leaf_mask)
    clean = cv2.bitwise_or(th, leaf_mask_inv)

    # Base name
    name, ext = os.path.splitext(fname)

    # Apply rotations + contrast
    for angle in rotation_angles:
        if angle == 90:
            rotated = cv2.rotate(clean, cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            rotated = cv2.rotate(clean, cv2.ROTATE_180)
        else:
            rotated = clean.copy()

        for cf in contrast_factors:
            contrasted = cv2.convertScaleAbs(rotated, alpha=cf, beta=0)
            out_name = f"{name}_r{angle}_c{str(cf).replace('.', '')}.jpg"
            out_path = os.path.join(output_folder, out_name)
            if cv2.imwrite(out_path, contrasted):
                print(f"Saved: {out_path}")
            else:
                print(f"Failed to save: {out_path}")


Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160302_r0_c10.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160302_r0_c13.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160302_r90_c10.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160302_r90_c13.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160302_r180_c10.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160302_r180_c13.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160313_r0_c10.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160313_r0_c13.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160313_r90_c10.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\IMG_20250420_160313_r90_c13.jpg
Saved: C:\Users\Hardika\Desktop\NNpro1_7species\unseen_leaves\